- 원본 데이터: newsdata/
    - ‘Content’ 컬럼: 뉴스 본문 원본 그대로
    - ‘Content_split’ 컬럼: ‘Content’ 컬럼 데이터를 kss 라이브러리로 문장 분리한 결과입니다.
- 전처리 이후 데이터: preprocessed_data/
    - ‘Sent’ 컬럼이 전처리 이후 기사 본문이고, 나머지는 기사 제목, 랭킹, 조회수 등 메타데이터입니다.
    - 언론사별 상위 20개씩 정렬되어있습니다. (ex. MBC 1-20위 기사 -> KBS 1-20위 기사 -> SBS 1-20위 기사)
    - 넉넉하게 8/1 ~ 10/13 기간 동안 방송 3사의 상위 20개 기사를 수집했습니다. 총 74개의 파일, (31+30+13)x20x3=4440개의 기사입니다.


In [ ]:
### kss: 문장 분리에 사용
### newspaper3k: 뉴스 기사 크롤링에 사용

# !pip install kss
# !pip install newspaper3k

## 1. 네이버 뉴스 크롤링 - 언론사별

- 네이버 뉴스 언론사별 뉴스
- 매일 언론사별 랭킹 상위 20개씩 수집(MBC, KBS, SBS)

In [2]:
import os
import re
import kss
import json
import glob
import time
import requests
import pandas as pd
from newspaper import Article
from bs4 import BeautifulSoup
from pathlib import Path

press_id = {"MBC":"214", "KBS":"056", "SBS":"055"}
# press_id = {"MBC":"214", "KBS":"056", "SBS":"055", "JTBC":"437"}

In [ ]:
def get_ranking_news(save_dir, date, n=1):
    total_time = 0
    news_list = []
    
    
    #언론사
    for press in press_id:
        start = time.time()
        url = "https://news.naver.com/main/ranking/office.nhn?officeId=" + press_id[press] + "&date=" + str(date)
        headers = {"User-Agent": "Mozilla/5.0(Windows NT 10.0;Win64;x64)AppleWebKit/537.36(KHTML, like Gecko)Chrome/87.0.4280.88 Safari/537.36"}
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, "html.parser")
        
        ranking_box = soup.find_all(class_="rankingnews_box_inner")
        
        
        #조회수(0) -> 댓글수(1)
        for ranking_type in range(1):
            ranking = ranking_box[ranking_type].find_all(class_="list_ranking_num")
            url_list = ranking_box[ranking_type].find_all(class_="list_content")
            
            # 랭킹순 상위 n개
            for rank in range(n):
                d = {}
                d['Date'] = int(date)
                d['Press'] = press
                d['Rank'] = ranking[rank].get_text()
                d['URL'] = url_list[rank].find('a')['href']
                d['Title'] = url_list[rank].find('a').get_text()
                if(ranking_type == 0):
                    d['View'] = url_list[rank].find(class_="list_view").get_text()
                elif(ranking_type==1):
                    d['Comment'] = url_list[rank].find(class_="list_comment nclicks('RBP.dcmtnwscmt')").get_text()
                news_list.append(d)
        
        # 본문 가져오기
        for news in news_list:
            resp = requests.get("https://news.naver.com" + news['URL'], headers=headers)
            news_url = "https://news.naver.com" + news['URL']
            article = Article(news_url, language='ko')
            article.download()
            article.parse()
            contents = article.text
            contents_split = article.text.split('\n')
            
            news['Content'] = contents
            news['Content_split'] = contents_split
#             print('contents:', contents)
#             print('contents_split:', contents_split)
#             print(len(contents_split))

    news_data = pd.DataFrame(news_list)   
    save_path = os.path.join(save_dir, 'ranking_' + str(date) + '.json')
    news_data.to_json(save_path, orient='table', index=False, force_ascii=False, indent=4)
    

    end=time.time()
    total_time += end - start
    print("Saved:" + save_path)
    print("Total time:", total_time)
    print("Average time", total_time/len(press_id))
    print("--------------------------------------------")
    

In [ ]:
# # 10월 상위 20개 뉴스
# days = 13
# topk = 20
# date = 20221001


# # 9월 상위 20개 뉴스
# days = 30
# topk = 20
# date = 20220901


# 8월 상위 20개 뉴스
days = 31
topk = 20
date = 20220801


save_dir = '/workspace/chitchat_cb/newsdata'

for _ in range(days):
    raw_data = get_ranking_news(save_dir, date, n=topk)
    date += 1

## 2. 전처리
- [참고 코드](https://github.com/HanNayeoniee/boostcamp/blob/main/week10-KLUE/(2%EA%B0%95)%20%EC%9E%90%EC%97%B0%EC%96%B4%EC%9D%98%20%EC%A0%84%EC%B2%98%EB%A6%AC%20-%200_%ED%95%9C%EA%B5%AD%EC%96%B4%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb)
- private repo이므로 권한 요청하기

In [3]:
### kss 라이브러리 사용해 문장 분리
def split_sentence(data):
    sents = []
    for sent in data:
        split_sent = kss.split_sentences(sent)
        sents.extend(split_sent)

    return sents


def remove_pattern(texts):
    outs = []
    p1 = re.compile(r'[\w\.-]+@[\w\.-]+')  # 이메일(영어@영어)
    p2 = re.compile(r'@[\w\.-]+')  # (@영어)
    p3 = re.compile('\d{2,3}-\d{3,4}-\d{4}$')  # 일반 전화번호
    p4 = re.compile('\d{3}-\d{3,4}-\d{4}$')  # 휴대폰번호
    p5 = re.compile('^◀')
    p6 = re.compile('^▷')
    p7 = re.compile('^=')
    p8 = re.compile('^MBC뉴스')
    p9 = re.compile('^영상제공 : ')
    # kbs
    p10 = re.compile('^KBS 뉴스')
    p11 = re.compile('^영상편집:')
    p12 = re.compile('^촬영기자:')
    p13 = re.compile('^\(영상취재 :')
    

    for text in texts:    
        res1 = p1.findall(text)
        res2 = p2.findall(text)
        res3 = p3.findall(text)
        res4 = p4.findall(text)
        res5 = p5.findall(text)
        res6 = p6.findall(text)
        res7 = p7.findall(text)
        res8 = p8.findall(text)
        res9 = p9.findall(text)
        res10 = p10.findall(text)
        res11 = p11.findall(text)
        res12 = p12.findall(text)
        res13 = p13.findall(text)
        
        if not res1 and not res2 and not res3 and not res4 and not res5 and not res6 and not res7 and not res8 \
            and not res9 and not res10 and not res11 and not res12 and not res13:
            outs.append(text)

    return outs


def remove_stops(texts):
    outs = []
    stop_mbc = ['MBC 뉴스는 24시간 여러분의 제보를 기다립니다.', '[뉴스투데이]', '[탐사기획 스트레이트]']
    
    for text in texts:    
        if text not in stop_mbc:
            outs.append(text)

    return outs

In [4]:
def remove_press(texts):
    patterns = [r"\(사진=[가-힣]{3,4}\)$",  # (사진=연합뉴스)
                    r"\(사진=[가-힣]{3,4} [가-힣]{3,4}\)$",  # (사진=온라인 커뮤니티)
                    r"(촬영기자:|영상편집:|그래픽:|영상취재:|편집:)[가-힣]{2,4}",  # 촬영기자:윤대민/영상편집:최근혁/그래픽:김석훈
                    r"(촬영기자: |영상편집: |그래픽: |영상취재: |편집: )[가-힣]{2,4}",  # 촬영기자: 윤대민/영상편집: 최근혁/그래픽: 김석훈
                    r"(영상취재|영상편집|그래픽|영상취재|편집)·(영상취재|영상편집|그래픽|영상취재|편집): [가-힣]{2,4}",  # 영상취재·편집: 위동원 
                    r"(영상취재|영상편집|그래픽|영상취재|편집)·(영상취재|영상편집|그래픽|영상취재|편집):[가-힣]{2,4}",  # 영상취재·편집:위동원 
                    r"\((영상취재 : |영상편집 : |그래픽 : |편집: )[가-힣]{2,4}·[가-힣]{2,4}", # (영상취재 : 김균종·조창현
                    r"(영상취재 : |영상편집 : |그래픽 : |편집 : \))[가-힣]{2,4}",  # 영상편집 : 박지인
                    r"\((영상취재 : |영상편집 : |그래픽 : |편집 : \))[가-힣]{2,4}",  # (영상편집 : 박지인
                    r"\[탐사기획 스트레이트]",
                    r"\[설문 참여하기]",
                    r"<기자>",
                    r"<앵커>",
                    r"앵커>"
                    ]
    
    outs = []
    for text in texts:
        for pat in patterns:
            text = re.sub(pat, "", text).strip()
        if text:
            outs.append(text)    
    return outs

In [5]:
def remove_url(texts):
    """
    URL을 제거합니다.
    ``주소: www.naver.com`` -> ``주소: ``
    """
    outs = []
    for text in texts:
        text = re.sub(r"(http|https)?:\/\/\S+\b|www\.(\w+\.)+\S*", "", text).strip()
        text = re.sub(r"pic\.(\w+\.)+\S*", "", text).strip()
        if text:
            outs.append(text)
    return outs


def filter(texts):
    outs = []
    for text in texts:
        text = re.sub('[▲━■▶◀△☎■▲※🎧]', '', text)
        if text:
            outs.append(text)
    return outs

In [8]:
## 모든 파일에 대해 전처리 수행
## 여기 기존 코드 날아갔음

# target = './newsdata/*.json'
target = './test/*.json'
json_list = glob.glob(target)
print('json파일 개수:', len(json_list))


for file in json_list:
    with open(file, 'r') as f:
        json_data = json.load(f)["data"]
        df = pd.DataFrame(json_data)


        sents = []
        for data in df["content"]:
            split_sent = split_sentence(data)
            print('문장 개수:', len(split_sent))
            outs = remove_pattern(split_sent)
            outs = remove_stops(outs)
            outs = remove_press(outs)
            outs = remove_url(outs)
            outs = filter(outs)        
            sents.append(split_sent)


#     df['sent'] = sents
    df['sent'] = sents
#     df = df[['Date', 'Press', 'Rank', 'URL', 'Title', 'View', 'Sent']]
    save_path = os.path.join('./test2/', Path(file).stem + '.json')
    df.to_json(save_path, orient='table', index=False, force_ascii=False, indent=4)
    print('Save to:', save_path)

json파일 개수: 3
문장 개수: 9
문장 개수: 43
문장 개수: 8
문장 개수: 7
문장 개수: 3
문장 개수: 4
문장 개수: 10
문장 개수: 12
문장 개수: 10
문장 개수: 23
문장 개수: 16
문장 개수: 22
문장 개수: 15
문장 개수: 12
문장 개수: 12
문장 개수: 3
문장 개수: 16
문장 개수: 4
문장 개수: 15
문장 개수: 20
Save to: ./test2/20220802.json
문장 개수: 9
문장 개수: 8
문장 개수: 51
문장 개수: 10
문장 개수: 3
문장 개수: 10
문장 개수: 14
문장 개수: 11
문장 개수: 9
문장 개수: 4
문장 개수: 10
문장 개수: 11
문장 개수: 17
문장 개수: 8
문장 개수: 9
문장 개수: 8
문장 개수: 13
문장 개수: 8
문장 개수: 17
문장 개수: 24
Save to: ./test2/20220803.json
문장 개수: 2
문장 개수: 10
문장 개수: 3
문장 개수: 3
문장 개수: 6
문장 개수: 21
문장 개수: 4
문장 개수: 3
문장 개수: 8
문장 개수: 9
문장 개수: 33
문장 개수: 23
문장 개수: 21
문장 개수: 8
문장 개수: 4
문장 개수: 3
문장 개수: 4
Save to: ./test2/20220801.json


In [ ]:
sent = ["세계 최초 버츄얼 휴먼 걸그룹인 이터니티가 사상 처음으로 TV 생방송에 출연했습니다.", "dkssud"]
split_sentence(sent)